In [ ]:
import mne
mne.set_log_level('WARNING')
import numpy as np
import matplotlib.pyplot as plt
from prep_function import (
    load_muse_data, base_filtering,
    median_filter_artifact_removal, auto_artifact_rejection, run_ica, annotate_ica_artifacts, annotate_dynamic_p2p
)
from prep_output import (
    plot_annotated_eeg, extract_frequency_bands,
    export_cleaned_data, plot_global_brainwaves,
    verify_against_reference, export_all_data_to_csv
)
import os

In [2]:
csv_path = r"C:\Users\shakeda\Downloads\mindMonitor_2025-05-23--21-30-11.csv"
output_dir = "0307_outputs"
os.makedirs(output_dir, exist_ok=True)

## Step 1: loading and interpolating nan values

In [ ]:
raw = load_muse_data(csv_path)

Precentage of NaN values in EEG data: 5.42%


In [4]:
raw.plot()

## Step 2: filtering 

In [5]:
base_filtering(raw)
print(raw.info)

<Info | 8 non-empty values
 bads: []
 ch_names: AF7, AF8, TP9, TP10
 chs: 4 EEG
 custom_ref_applied: True
 highpass: 1.0 Hz
 lowpass: 40.0 Hz
 meas_date: unspecified
 nchan: 4
 projs: []
 sfreq: 256.0 Hz
>


In [8]:
raw.plot()

## Step 3: auto-annotating artifacts

In [12]:
raw = annotate_dynamic_p2p(raw, n_mads=8, win_sec=0.05, step_sec=0.01)

In [13]:
raw.plot()

<div style="
     border: 2px solid #d9534f;
     background-color: #f2dede;
     color: #a94442;
     padding: 10px 15px;
     border-radius: 4px;
     margin: 10px 0;
">
<strong>⚠️ IMPORTANT:</strong> To experiment with new parameters:
  <ol>
    <li>Run the cell below to reset your annotations.</li>
    <li>Update the parameters in the function <em>annotate_dynamic_p2p</em> above.</li>
    <li>Re-run the cell above with your new settings.</li>
  </ol>
</div>

In [11]:
# NOTICE: this line resets the annotations.
_ = raw.set_annotations(raw.annotations[np.array(raw.annotations.description) != 'BAD_dynamic'])

## Step 4: performing ICA

In [ ]:
# Call ICA

## Step 5: extract frequency bands per channel

In [14]:
per_channel_raws = extract_frequency_bands(raw)

In [ ]:
per_channel_raws['TP9'].plot()

## Step 6: save the data

In [22]:
for curr_chan, curr_raw in per_channel_raws.items():
    curr_raw.save(os.path.join(output_dir, f"{curr_chan}_decomposed_raw.fif"))

In [ ]:
csv_path = f'{output_dir}/all_channels_with_bands.csv'
export_all_data_to_csv(raw, csv_path)

Saved combined data to 0307_outputs/all_channels_with_bands.csv
